# Simulating EPA-RE using points of low-order

In [ ]:
import pickle
import itertools
import glob

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

from pathlib import Path
from random import randint, randbytes
from typing import Type, Any

from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange

from pyecsca.ec.params import DomainParameters, get_params
from pyecsca.ec.mult import *
from pyecsca.sca.re.rpa import multiples_computed
from pyecsca.misc.utils import TaskExecutor

from common import *

## Initialize

In [ ]:
print(len(all_mults))

In [ ]:
# Needs imports on the inside to be spawn enabled to save memory.

def get_general_multiples(bits: int, samples: int = 1000) -> MultResults:
    from random import randint
    results = []
    for _ in range(samples):
        big_scalar = randint(1, 2**bits)
        results.append({big_scalar})
    return MultResults(results, samples)

def get_general_n_multiples(bits: int, n: int, samples: int = 1000) -> MultResults:
    from random import randint
    results = []
    for _ in range(samples):
        smult = set()
        for i in range(n):
            b = randint(1,256)
            smult.add(randint(2**b,2**(b+1)))
        results.append(smult)
    return MultResults(results, samples)

def get_small_scalar_multiples(mult: MultIdent,
                               params: DomainParameters,
                               bits: int,
                               samples: int = 1000,
                               use_init: bool = True,
                               use_multiply: bool = True,
                               seed: bytes | None = None) -> MultResults:
    from pyecsca.sca.re.rpa import multiples_computed
    import random
    
    results = []
    if seed is not None:
        random.seed(seed)

    # If no countermeasure is used, we have fully random scalars.
    # Otherwise, fix one per chunk.
    if mult.countermeasure is None:
        scalars = [random.randint(1, 2**bits) for _ in range(samples)]
    else:
        one = random.randint(1, 2**bits)
        scalars = [one for _ in range(samples)]

    for scalar in scalars:
        # Use a list for less memory usage.
        results.append(list(multiples_computed(scalar, params, mult.klass, mult.partial, use_init, use_multiply)))
    return MultResults(results, samples)

## Prepare

In [ ]:
category = "secg"
curve = "secp256r1"
params = get_params(category, curve, "projective")
num_workers = 20
bits = params.order.bit_length()
samples = 1000
selected_mults = all_mults

## Run
Run this cell as many times as you want. It will write chunks into files.

In [ ]:
multiples_mults = {}
chunk_id = randbytes(4).hex()
with TaskExecutor(max_workers=num_workers, mp_context=spawn_context) as pool, enable_spawn(get_small_scalar_multiples) as target:
    for mult in selected_mults:
        for countermeasure in (None, "gsr", "additive", "multiplicative", "euclidean"):
            mwc = mult.with_countermeasure(countermeasure)
            pool.submit_task(mwc,
                             target,
                             mwc, params, bits, samples, seed=chunk_id)
    for mult, future in tqdm(pool.as_completed(), desc="Computing small scalar distributions.", total=len(pool.tasks)):
        print(f"Got {mult}.")
        if error := future.exception():
            print(error)
            continue
        res = future.result()
        if mult not in multiples_mults:
            multiples_mults[mult] = res
        else:
            # Accumulate
            multiples_mults[mult].merge(res)
    # Handle the enable_spawn trick that messes up class modules.
    for k, v in multiples_mults.items():
        v.__class__ = MultResults
        v.__module__ = "common"
with open(f"multiples_{category}_{curve}_{bits}_ctr_chunk{chunk_id}.pickle","wb") as h:
    pickle.dump(multiples_mults, h)

### Load
Beware, the following load with try to load all chunks into memory, that will be very large.

In [ ]:
multiples_mults = {}
for fname in glob.glob(f"multiples_{category}_{curve}_{bits}_chunk*.pickle"):
    with open(fname, "rb") as f:
        multiples_loaded = pickle.load(f)
        for mult, vals in multiples_loaded.items():
            if mult not in multiples_mults:
                multiples_mults[mult] = vals
            else:
                multiples_mults[mult].merge(vals)